Reading File Data to DataFrame

In [1]:
#import module
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#create session in order to be capable of accessing all Spark API
spark = SparkSession \
    .builder \
    .appName("Introduction to Spark DataFrame") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#define data schema for file we want to read
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])    

# read csv file with our defined schema into Spark DataFrame, and use "tab" delimiter
purchaseDataframe = spark.read.csv(
    "dataset/purchases.csv", 
    header=True, schema=purchaseSchema, sep="\t")
#show 3 rows of our DataFrame
purchaseDataframe.show(3)

+----------+-----+----------+----------------+------+-------+
|      Date| Time|      City|            Item| Total|Payment|
+----------+-----+----------+----------------+------+-------+
|2012-01-01|09:00|  San Jose|  Men's Clothing|214.05|   Amex|
|2012-01-01|09:00|Fort Worth|Women's Clothing|153.57|   Visa|
|2012-01-01|09:00| San Diego|           Music| 66.08|   Cash|
+----------+-----+----------+----------------+------+-------+
only showing top 3 rows



Counting number of rows, printing dataframe schema and printing statistic of our data

In [2]:
#count number of rows of our dataFrame
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)
#show our dataFrame schema
purchaseDataframe.printSchema()
#show statistic of the data we want
purchaseDataframe.describe('Total').show()

number of rows:  4138476
root
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Total: float (nullable = true)
 |-- Payment: string (nullable = true)

+-------+------------------+
|summary|             Total|
+-------+------------------+
|  count|           4138476|
|   mean|249.96108549398525|
| stddev| 144.3174111542959|
|    min|               0.0|
|    max|            499.99|
+-------+------------------+



Creating a new dataFrame from a subset of our existing dataFrame

In [3]:
#create new dataFrame from "City" and "Total" columns
newDataframe = purchaseDataframe.select(purchaseDataframe['City'], 
                                              purchaseDataframe['Total'])
newDataframe.show(3); #menampilkan 3 baris DataFrame baru kita
newDataframe.printSchema() #print skema dari DataFrame baru kita

+----------+------+
|      City| Total|
+----------+------+
|  San Jose|214.05|
|Fort Worth|153.57|
| San Diego| 66.08|
+----------+------+
only showing top 3 rows

root
 |-- City: string (nullable = true)
 |-- Total: float (nullable = true)



Adding a constant value to every row data in certain column

In [4]:
#adding constant value of 10 to every row data in "Total" column
purchaseDataframe.select(purchaseDataframe['City'],
                         purchaseDataframe['Total']+10).show(3)

+----------+------------+
|      City|(Total + 10)|
+----------+------------+
|  San Jose|      224.05|
|Fort Worth|      163.57|
| San Diego|       76.08|
+----------+------------+
only showing top 3 rows



Filtering dataFrame using our defined condition

In [5]:
#filter only row data whose "Total" column value > 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(3)

+----------+-----+----------+-------------------+------+----------+
|      Date| Time|      City|               Item| Total|   Payment|
+----------+-----+----------+-------------------+------+----------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|
+----------+-----+----------+-------------------+------+----------+
only showing top 3 rows



In [6]:
#filter only row data whose "Total" column value > 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).count()

2482214

Sorting dataFrame by certain column

In [7]:
sortedByCity = purchaseDataframe.orderBy('City').show(10)

+----------+-----+-----------+-------------------+------+----------+
|      Date| Time|       City|               Item| Total|   Payment|
+----------+-----+-----------+-------------------+------+----------+
|2012-10-07|11:26|Albuquerque|   Women's Clothing| 25.53|      Amex|
|2012-10-07|12:17|Albuquerque|   Women's Clothing|  94.2|  Discover|
|2012-10-07|11:40|Albuquerque|               Toys|299.63|MasterCard|
|2012-10-07|11:18|Albuquerque|             Crafts|475.77|      Visa|
|2012-10-07|11:41|Albuquerque|              Music|365.64|      Visa|
|2012-10-07|10:39|Albuquerque|       Pet Supplies| 401.3|MasterCard|
|2012-10-07|11:49|Albuquerque|Children's Clothing|205.83|      Amex|
|2012-10-07|10:43|Albuquerque|            Cameras|309.07|  Discover|
|2012-10-07|11:53|Albuquerque|     Men's Clothing|219.69|MasterCard|
|2012-10-07|11:54|Albuquerque|  Health and Beauty| 480.4|MasterCard|
+----------+-----+-----------+-------------------+------+----------+
only showing top 10 rows



In [8]:
#import col,asc.desc
from pyspark.sql.functions import col,asc,desc
sortedByTotal = purchaseDataframe.orderBy(col('City').asc(),col('Total').desc()).show(10)

+----------+-----+-----------+--------------------+------+----------+
|      Date| Time|       City|                Item| Total|   Payment|
+----------+-----+-----------+--------------------+------+----------+
|2012-09-22|11:01|Albuquerque|        Pet Supplies|499.98|      Visa|
|2012-01-27|15:33|Albuquerque|           Computers|499.98|      Visa|
|2012-03-03|17:03|Albuquerque|Consumer Electronics|499.97|MasterCard|
|2012-08-27|16:34|Albuquerque|Consumer Electronics|499.93|      Visa|
|2012-04-30|16:01|Albuquerque|Consumer Electronics|499.91|      Cash|
|2012-12-01|10:15|Albuquerque|                 CDs|499.89|      Visa|
|2012-08-12|10:25|Albuquerque| Children's Clothing|499.89|  Discover|
|2012-04-04|13:00|Albuquerque|           Computers|499.87|      Cash|
|2012-08-04|17:24|Albuquerque|                DVDs|499.85|      Visa|
|2012-03-13|10:46|Albuquerque|   Health and Beauty|499.85|  Discover|
+----------+-----+-----------+--------------------+------+----------+
only showing top 10 

Calculating number of transactions in each city

In [9]:
numTransactionEachCity = purchaseDataframe.groupBy("City").count()
numTransactionEachCity.orderBy("City").show(5)

+-----------+-----+
|       City|count|
+-----------+-----+
|Albuquerque|40345|
|    Anaheim|40086|
|  Anchorage|39806|
|  Arlington|40348|
|    Atlanta|40168|
+-----------+-----+
only showing top 5 rows



Indexing and Accessing DataFrame

In [10]:
#import monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id

newPurchasedDataframe = purchaseDataframe.withColumn(
    "index", monotonically_increasing_id())
newPurchasedDataframe.show(7)
row2Till4 = newPurchasedDataframe.filter((newPurchasedDataframe['index']>=2) &
                                         (newPurchasedDataframe['index']<=4))
row2Till4.show()

+----------+-----+----------+-------------------+------+----------+-----+
|      Date| Time|      City|               Item| Total|   Payment|index|
+----------+-----+----------+-------------------+------+----------+-----+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|    0|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|153.57|      Visa|    1|
|2012-01-01|09:00| San Diego|              Music| 66.08|      Cash|    2|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|    3|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|    4|
|2012-01-01|09:00|  Stockton|     Men's Clothing|247.18|MasterCard|    5|
|2012-01-01|09:00|    Austin|            Cameras| 379.6|      Visa|    6|
+----------+-----+----------+-------------------+------+----------+-----+
only showing top 7 rows

+----------+-----+----------+-------------------+------+----------+-----+
|      Date| Time|      City|               Item| Total|   Payment|index|
+----------+-

In [11]:
dataRow2ColumnTotal = newPurchasedDataframe.filter(newPurchasedDataframe['index']==2).select('Total')
dataRow2ColumnTotal.show()

+-----+
|Total|
+-----+
|66.08|
+-----+



Using SQL query in dataFrame

In [12]:
#we need to make sql temporary view for our dataFrame
newPurchasedDataframe.createOrReplaceTempView("purchaseSql")

#select "Total" dan "Payment" column from our sql temporary view
anotherNewDataframe = spark.sql("SELECT Total, Payment FROM purchaseSql")
anotherNewDataframe.show(3)

+------+-------+
| Total|Payment|
+------+-------+
|214.05|   Amex|
|153.57|   Visa|
| 66.08|   Cash|
+------+-------+
only showing top 3 rows



In [13]:
#sorting data by "City" column alphabetically and "index" colomn ascendingly
orderByCity = spark.sql("SELECT * FROM purchaseSql ORDER BY City, index ASC")
orderByCity.show(10)

+----------+-----+-----------+-----------------+------+----------+-----+
|      Date| Time|       City|             Item| Total|   Payment|index|
+----------+-----+-----------+-----------------+------+----------+-----+
|2012-01-01|09:03|Albuquerque|     Pet Supplies| 440.7|      Cash|   85|
|2012-01-01|09:08|Albuquerque|     Pet Supplies|484.24|  Discover|  183|
|2012-01-01|09:09|Albuquerque|           Crafts|149.94|      Cash|  196|
|2012-01-01|09:15|Albuquerque|Health and Beauty|192.48|      Amex|  325|
|2012-01-01|09:15|Albuquerque|            Music| 305.0|MasterCard|  334|
|2012-01-01|09:16|Albuquerque|          Cameras|326.11|      Amex|  353|
|2012-01-01|09:18|Albuquerque|        Computers| 233.1|      Cash|  384|
|2012-01-01|09:19|Albuquerque|        Computers|290.43|MasterCard|  415|
|2012-01-01|09:29|Albuquerque|           Garden| 91.31|      Cash|  619|
|2012-01-01|09:29|Albuquerque|Health and Beauty|183.44|      Cash|  624|
+----------+-----+-----------+-----------------+---

In [14]:
#filter Total>200 and Richmond order by Payment
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE (Total>200 and City='Richmond') ORDER BY Payment")
filterAndSortWithSQL.show(4)

+----------+-----+--------+--------------------+------+-------+-----------+
|      Date| Time|    City|                Item| Total|Payment|      index|
+----------+-----+--------+--------------------+------+-------+-----------+
|2012-10-07|12:11|Richmond|Consumer Electronics|329.74|   Amex|25769803790|
|2012-10-07|15:41|Richmond|Consumer Electronics|398.89|   Amex|25769803811|
|2012-10-07|12:55|Richmond|                DVDs|275.32|   Amex|25769803798|
|2012-10-07|10:51|Richmond|                DVDs|332.64|   Amex|25769803780|
+----------+-----+--------+--------------------+------+-------+-----------+
only showing top 4 rows

